In [7]:
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import copy
import missingno as msno
%matplotlib inline 
# 기본 경로 설정
os.chdir('C:/Users/gksru/Desktop/study/Lpoint/제6회 L.POINT Big Data Competition')
# 단위 표시 설정
pd.options.display.float_format = '{:.5f}'.format
# 폰트 설정
mpl.rc('font', family=' Malgun Gothic')
# 유니코드에서  음수 부호설정
mpl.rc('axes', unicode_minus=False)

warnings.filterwarnings('ignore')

In [8]:
online_data = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv')
online_data.head()

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm
0,7809,1,8,5,A03,20190913,01:16,2571103,nan,NaN,34.00000,2663.00000,DIRECT,NaN
1,7809,1,4,2,A03,20190913,01:14,2485909,nan,NaN,34.00000,2663.00000,DIRECT,NaN
2,7809,1,11,5,A03,20190913,01:17,2646597,nan,NaN,34.00000,2663.00000,DIRECT,NaN
3,7809,1,1,2,A03,20190913,00:46,788304,nan,NaN,34.00000,2663.00000,DIRECT,NaN
4,7809,1,9,5,A03,20190913,01:17,2617609,nan,NaN,34.00000,2663.00000,DIRECT,NaN


In [9]:
trade_data = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv')
trade_data.head()

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
0,21922,104999,1,A03,unknown,20190920,12:41,5990,1
1,21279,104907,4,A03,unknown,20190920,10:27,10900,1
2,39423,105124,11,A03,unknown,20190920,17:26,12900,1
3,18362,104010,1,A03,unknown,20190920,09:57,9900,1
4,39423,105124,13,A03,0565,20190920,17:26,2990,1


In [10]:
demo_data = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv')
demo_data.head()

,clnt_id,clnt_gender,clnt_age
0,1,unknown,unknown
1,2,F,30
2,3,unknown,unknown
3,4,unknown,unknown
4,5,unknown,unknown


In [11]:
class_data = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv')
class_data.head()

,pd_c,clac_nm1,clac_nm2,clac_nm3
0,1,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Automobile Oil / Additives
1,2,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Lights
2,3,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Paint
3,4,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Filters
4,5,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Wiper Blades


In [12]:
online_data['sess_dt'] = online_data['sess_dt'].astype('str')
online_data['sess_dt'] = online_data['sess_dt'] + online_data['hit_tm']
online_data['sess_dt'] = online_data['sess_dt'].apply(lambda x: datetime.datetime.strptime(x, '%Y%m%d%H:%M'))

In [13]:
online_data

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm
0,7809,1,8,5,A03,2019-09-13 01:16:00,01:16,2571103,nan,NaN,34.00000,2663.00000,DIRECT,NaN
1,7809,1,4,2,A03,2019-09-13 01:14:00,01:14,2485909,nan,NaN,34.00000,2663.00000,DIRECT,NaN
2,7809,1,11,5,A03,2019-09-13 01:17:00,01:17,2646597,nan,NaN,34.00000,2663.00000,DIRECT,NaN
3,7809,1,1,2,A03,2019-09-13 00:46:00,00:46,788304,nan,NaN,34.00000,2663.00000,DIRECT,NaN
4,7809,1,9,5,A03,2019-09-13 01:17:00,01:17,2617609,nan,NaN,34.00000,2663.00000,DIRECT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196357,31040,1,1,0,A02,2019-08-15 03:47:00,03:47,13525,nan,다이슨 에어랩,4.00000,62.00000,unknown,mobile_web
3196358,4129,1,2,0,A02,2019-07-21 01:22:00,01:22,250099,nan,여성메탈시계,3.00000,250.00000,unknown,mobile_web
3196359,4129,1,1,0,A02,2019-07-21 01:18:00,01:18,1525,nan,여성메탈시계,3.00000,250.00000,unknown,mobile_web
3196360,54403,3,1,0,A02,2019-07-31 20:14:00,20:14,16905,nan,비비고왕교자,3.00000,68.00000,unknown,mobile_app
